<a href="https://colab.research.google.com/github/yongsa-nut/SF323_CN408_AIEngineer/blob/main/SF323_CN408_Tool_Use_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tool Use Demo

## 0. Setting up

In [ ]:
from openai import OpenAI
from google.colab import userdata
import json

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=userdata.get('openrouter'),
)

In [ ]:
completion = client.chat.completions.create(
  model="google/gemini-2.5-flash-lite",
  messages=[
    {
      "role": "user",
      "content": "Hello"
    }
  ]
)
print(completion.choices[0].message.content)

Hello! How can I help you today?


Asking a question about info that models don't know

In [ ]:
completion = client.chat.completions.create(
  model="google/gemini-2.5-flash-lite",
  messages=[
    {
      "role": "user",
      "content": "What is a current temperature in Bangkok?"
    }
  ]
)
print(completion.choices[0].message.content)

I cannot provide real-time, current weather information. My knowledge cutoff is **June 2024**, and dynamic data like live temperatures is outside of that scope.

To get the current temperature in Bangkok, you can check a reliable weather source such as:

*   **Weather websites:** AccuWeather, The Weather Channel, BBC Weather, etc.
*   **Your smartphone's built-in weather app.**
*   **A quick Google search for "temperature Bangkok".**


## 1. Tool Use (Function Calling)

Openrouter Tool Calling: https://openrouter.ai/docs/features/tool-calling

See also: https://platform.openai.com/docs/guides/function-calling?api-mode=chat

**Key idea**: The model calls tools by outputing tool names and arguments. Dev then run the tools with those arguments and add the tool results back to the model.

**Note**: Some models on openrouter may not have providers allow tool calling (e.g. Qwen3-8b)

**Note 2**: Under the hood, functions are injected into the system message in a syntax the model has been trained on. This means functions count against the model's context limit and are billed as input tokens.

### Weather Tool

Define the weather tool

In [ ]:
weather_tools = [{
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current temperature (Celcius) for a given location.",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "City or Country e.g. New York, Colombia, France"
                }
            },
            "required": ["location"]
        }
    }
}]

Create ad-hoc `get_weather` function

In [ ]:
def get_weather(city):
  return 32

Test the tool

In [ ]:
messages = [
    {
      "role": "user",
      "content": "What is a current temperature in Bangkok?"
    }
]

completion = client.chat.completions.create(
    model = "google/gemini-2.5-flash-lite",
    tools = weather_tools,                                      # Add tools
    messages = messages
)
print(completion.choices[0].message.content)

Check the finish reason

In [ ]:
print(completion.choices[0])

Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content='', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='tool_0_get_weather', function=Function(arguments='{"location":"Bangkok"}', name='get_weather'), type='function', index=0)], reasoning=None), native_finish_reason='STOP')


Get the argument

In [ ]:
print(completion.choices[0].message.tool_calls[0].function.arguments)

{"location":"Bangkok"}


Call the tool and append back the tool result

In [ ]:
# Call our function
tool_call = completion.choices[0].message.tool_calls[0]
arguments = json.loads(tool_call.function.arguments)
result = get_weather(arguments['location'])  # our function

# First append the tool calling message
messages.append(completion.choices[0].message)  # tool calling
# Then append the tool result
messages.append({                               # append result message
    "role": "tool",
    "tool_call_id": tool_call.id,
    "content": str(result)
})

# Call the model with everything
completion_2 = client.chat.completions.create(
    model="google/gemini-2.5-flash-lite",
    messages=messages,
    tools=weather_tools,
)
print(completion_2.choices[0].message.content)

The current temperature in Bangkok is 32°C.


In [ ]:
print(completion_2.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The current temperature in Bangkok is 32 degrees Celsius.', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None, reasoning=None), native_finish_reason='STOP')


Put everything together

In [ ]:
# Complete one step
messages = [
    {
      "role": "user",
      "content": "What is a current temperature in Bangkok?"  # Test with multiple cities (flash-lite can't handle multiple function calling)
    }
]

# 1. User's query
completion = client.chat.completions.create(
    model = "google/gemini-2.5-flash",
    tools = weather_tools,                          # Add tools
    messages = messages
)

if completion.choices[0].finish_reason == 'tool_calls':

    # 2. Get the tool call
    tool_calls = completion.choices[0].message.tool_calls
    # Don't forget to add the message back!!
    messages.append(completion.choices[0].message)

    # 3. Run the tools
    for tool_call in tool_calls:
        arguments = json.loads(tool_call.function.arguments)
        result = get_weather(arguments['location'])

        # 4. Add the results back
        messages.append({
            "role": "tool",
            "tool_call_id": tool_call.id,
            "name":tool_call.function.name,
            "content": str(result),
        })

    # 5. Call the model with everything
    completion_2 = client.chat.completions.create(
        model="google/gemini-2.5-flash",
        messages=messages,
        tools=weather_tools,
    )
    print(completion_2.choices[0].message.content)

The current temperature in Bangkok is 32 Celsius.


In [ ]:
tool_calls

[ChatCompletionMessageToolCall(id='tool_0_get_weather', function=Function(arguments='{"location":"Bangkok"}', name='get_weather'), type='function', index=0)]

### Simple Customer Chatbot

- Can look up customer information, retrieve order details, and cancel orders on behalf of the custome.



First, let's create functions

In [ ]:
def get_customer_info(customer_id):
    # Simulated customer data
    customers = {
        "C1": {"name": "John Doe", "email": "john@example.com", "phone": "123-456-7890"},
        "C2": {"name": "Jane Smith", "email": "jane@example.com", "phone": "987-654-3210"}
    }
    return customers.get(customer_id, "Customer not found")

def get_order_details(order_id):
    # Simulated order data
    orders = {
        "O1": {"id": "O1", "product": "Widget A", "quantity": 2, "price": 19.99, "status": "Shipped"},
        "O2": {"id": "O2", "product": "Gadget B", "quantity": 1, "price": 49.99, "status": "Processing"}
    }
    return orders.get(order_id, "Order not found")

def cancel_order(order_id):
    # Simulated order cancellation
    if order_id in ["O1", "O2"]:
        return True
    else:
        return False

Then, let define tools

In [ ]:
customer_tools = [
    {
        "type": "function",
        "function":{
            "name": "get_customer_info",
            "description": ("Retrieves customer information based on their customer ID."
                             "Returns the customer's name, email, and phone number."),
            "parameters": {
                "type": "object",
                "properties": {
                    "customer_id": {
                        "type": "string",
                        "description": "The unique identifier for the customer."
                    }
                },
                "required": ["customer_id"]
            }
        }
    },
    {
        "type": "function",
        "function":{
            "name": "get_order_details",
            "description": ("Retrieves the details of a specific order based on the order ID."
                            "Returns the order ID, product name, quantity, price, and order status."),
            "parameters": {
                "type": "object",
                "properties": {
                    "order_id": {
                        "type": "string",
                        "description": "The unique identifier for the order."
                    }
                },
                "required": ["order_id"]
            }
        }
    },
    {
        "type": "function",
        "function":{
            "name": "cancel_order",
            "description": ("Cancels an order based on the provided order ID."
                            "Returns a confirmation message if the cancellation is successful."),
            "parameters": {
                "type": "object",
                "properties": {
                    "order_id": {
                        "type": "string",
                        "description": "The unique identifier for the order to be cancelled."
                    }
                },
                "required": ["order_id"]
            }
        }
    }
]

Helper function

In [ ]:
# a function to process the tool calls made by Claude and return the appropriate results
def process_tool_call(tool_name, tool_input):
    if tool_name == "get_customer_info":
        return get_customer_info(tool_input["customer_id"])
    elif tool_name == "get_order_details":
        return get_order_details(tool_input["order_id"])
    elif tool_name == "cancel_order":
        return cancel_order(tool_input["order_id"])

The main chatbot

In [ ]:
import json

def chatbot_interaction(user_message, model="google/gemini-2.5-flash-lite"):
    print(f"\n{'='*50}\nUser Message: {user_message}\n{'='*50}")

    messages = [
        {"role": "system", "content":"You are a helpful customer support agent. You must use tools to answer questions."},
        {"role": "user", "content": user_message}
    ]
    # Get the initial response
    response = client.chat.completions.create(
        model=model,
        messages=messages,                  # Message history
        tools=customer_tools,                        # Tool
        temperature = 1                     # set temp = 0 so that's the answer is mostly deterministic
    )
    llm_response = response.choices[0].message
    print(f"\nInitial Response:")
    print(f"Content: {llm_response}")

    if response.choices[0].finish_reason == 'tool_calls':
        messages.append(llm_response) # LLM response including tool call message
        for tool_call in llm_response.tool_calls:

            # Obtain the name and arguments from tool call message
            tool_name = tool_call.function.name
            tool_arguments = json.loads(tool_call.function.arguments)
            print(f"\nTool Used: {tool_name}")
            print(f"Tool arguments:")
            print(tool_arguments)

            tool_result = process_tool_call(tool_name, tool_arguments)
            print(f"\nTool Result:")
            print(tool_result)

            # Construct tool result messages
            tool_result = {
                "role" : "tool",
                "tool_call_id": tool_call.id,
                "name":tool_name,
                "content" : str(tool_result)
            }

            # Add tool call and tool result to messages
            messages.append(tool_result)

        print(f'\nFinal Response: ')
        response = client.chat.completions.create(
                model=model,
                messages=messages,
                tools=customer_tools,
            )
        print(response.choices[0].message.content)

In [ ]:
chatbot_interaction("Can you tell me the email address for customer C1?")

In [ ]:
chatbot_interaction("What is the status of order O2?")

In [ ]:
chatbot_interaction("Please cancel order O1 for me.")

## 2. Multiple tool uses

### Calendar Bot

Three tools:
- `get_current_datetime`: Get the current date time
- `add_duration_to_datetime`: Add duration to date time
- `set_reminder`:  Set a reminder

In [ ]:
# @title Functions

from datetime import datetime, timedelta

def get_current_datetime(date_format="%Y-%m-%d %H:%M:%S"):
    if not date_format:
        raise ValueError("date_format cannot be empty")
    return datetime.now().strftime(date_format)

def add_duration_to_datetime(
    datetime_str, duration=0, unit="days", input_format="%Y-%m-%d"
):
    date = datetime.strptime(datetime_str, input_format)

    if unit == "seconds":
        new_date = date + timedelta(seconds=duration)
    elif unit == "minutes":
        new_date = date + timedelta(minutes=duration)
    elif unit == "hours":
        new_date = date + timedelta(hours=duration)
    elif unit == "days":
        new_date = date + timedelta(days=duration)
    elif unit == "weeks":
        new_date = date + timedelta(weeks=duration)
    elif unit == "months":
        month = date.month + duration
        year = date.year + month // 12
        month = month % 12
        if month == 0:
            month = 12
            year -= 1
        day = min(
            date.day,
            [
                31,
                29
                if year % 4 == 0 and (year % 100 != 0 or year % 400 == 0)
                else 28,
                31,
                30,
                31,
                30,
                31,
                31,
                30,
                31,
                30,
                31,
            ][month - 1],
        )
        new_date = date.replace(year=year, month=month, day=day)
    elif unit == "years":
        new_date = date.replace(year=date.year + duration)
    else:
        raise ValueError(f"Unsupported time unit: {unit}")

    return new_date.strftime("%A, %B %d, %Y %I:%M:%S %p")


def set_reminder(content, timestamp):
    print(
        f"----\nSetting the following reminder for {timestamp}:\n{content}\n----"
    )

In [ ]:
# @title Function Schemas

add_duration_to_datetime_schema = {
    "type": "function",
    "function":{
        "name": "add_duration_to_datetime",
        "description": "Adds a specified duration to a datetime string and returns the resulting datetime in a detailed format. This tool converts an input datetime string to a Python datetime object, adds the specified duration in the requested unit, and returns a formatted string of the resulting datetime. It handles various time units including seconds, minutes, hours, days, weeks, months, and years, with special handling for month and year calculations to account for varying month lengths and leap years. The output is always returned in a detailed format that includes the day of the week, month name, day, year, and time with AM/PM indicator (e.g., 'Thursday, April 03, 2025 10:30:00 AM').",
        "parameters": {
            "type": "object",
            "properties": {
                "datetime_str": {
                    "type": "string",
                    "description": "The input datetime string to which the duration will be added. This should be formatted according to the input_format parameter.",
                },
                "duration": {
                    "type": "number",
                    "description": "The amount of time to add to the datetime. Can be positive (for future dates) or negative (for past dates). Defaults to 0.",
                },
                "unit": {
                    "type": "string",
                    "description": "The unit of time for the duration. Must be one of: 'seconds', 'minutes', 'hours', 'days', 'weeks', 'months', or 'years'. Defaults to 'days'.",
                },
                "input_format": {
                    "type": "string",
                    "description": "The format string for parsing the input datetime_str, using Python's strptime format codes. For example, '%Y-%m-%d' for ISO format dates like '2025-04-03'. Defaults to '%Y-%m-%d'.",
                },
            },
            "required": ["datetime_str"],
        },
    }
}

set_reminder_schema = {
    "type": "function",
    "function":{
        "name": "set_reminder",
        "description": "Creates a timed reminder that will notify the user at the specified time with the provided content. This tool schedules a notification to be delivered to the user at the exact timestamp provided. It should be used when a user wants to be reminded about something specific at a future point in time. The reminder system will store the content and timestamp, then trigger a notification through the user's preferred notification channels (mobile alerts, email, etc.) when the specified time arrives. Reminders are persisted even if the application is closed or the device is restarted. Users can rely on this function for important time-sensitive notifications such as meetings, tasks, medication schedules, or any other time-bound activities.",
        "parameters": {
            "type": "object",
            "properties": {
                "content": {
                    "type": "string",
                    "description": "The message text that will be displayed in the reminder notification. This should contain the specific information the user wants to be reminded about, such as 'Take medication', 'Join video call with team', or 'Pay utility bills'.",
                },
                "timestamp": {
                    "type": "string",
                    "description": "The exact date and time when the reminder should be triggered, formatted as an ISO 8601 timestamp (YYYY-MM-DDTHH:MM:SS) or a Unix timestamp. The system handles all timezone processing internally, ensuring reminders are triggered at the correct time regardless of where the user is located. Users can simply specify the desired time without worrying about timezone configurations.",
                },
            },
            "required": ["content", "timestamp"],
        },
    }
}

get_current_datetime_schema = {
    "type": "function",
    "function":{
        "name": "get_current_datetime",
        "description": "Returns the current date and time formatted according to the specified format string. This tool provides the current system time formatted as a string. Use this tool when you need to know the current date and time, such as for timestamping records, calculating time differences, or displaying the current time to users. The default format returns the date and time in ISO-like format (YYYY-MM-DD HH:MM:SS).",
        "parameters": {
            "type": "object",
            "properties": {
                "date_format": {
                    "type": "string",
                    "description": "A string specifying the format of the returned datetime. Uses Python's strftime format codes. For example, '%Y-%m-%d' returns just the date in YYYY-MM-DD format, '%H:%M:%S' returns just the time in HH:MM:SS format, '%B %d, %Y' returns a date like 'May 07, 2025'. The default is '%Y-%m-%d %H:%M:%S' which returns a complete timestamp like '2025-05-07 14:32:15'.",
                    "default": "%Y-%m-%d %H:%M:%S",
                }
            },
            "required": [],
        },
    }
}

calendar_tools = [add_duration_to_datetime_schema, set_reminder_schema, get_current_datetime_schema]

In [ ]:
# @title Tool running help Functions

def run_tool(tool_name, tool_input):
    if tool_name == "get_current_datetime":
        return get_current_datetime(**tool_input)
    elif tool_name == "add_duration_to_datetime":
        return add_duration_to_datetime(**tool_input)
    elif tool_name == "set_reminder":
        return set_reminder(**tool_input)

def run_tools(tool_calls):
    tool_results = []
    for tool_call in tool_calls:
        try:
            tool_name = tool_call.function.name
            tool_arguments = json.loads(tool_call.function.arguments)
            tool_result = run_tool(tool_name, tool_arguments)
            tool_result = {
                    "role" : "tool",
                    "tool_call_id": tool_call.id,
                    "name":tool_name,
                    "content" : str(tool_result)
            }
        except Exception as e:
            tool_result = {
                    "role" : "tool",
                    "tool_call_id": tool_call.id,
                    "name":tool_name,
                    "content" : f"Error: {e}",
                     #"is_error": True,         # some provider accept this
            }

        tool_results.append(tool_result)

    return tool_results

Here's what happens behind the scenes when a model needs multiple tools:

1. User asks: "What day is 103 days from today?"
2. Model responds with a tool use block requesting `get_current_datetime`
3. Your server calls the function and returns the result
4. Model realizes it needs more information and requests `add_duration_to_datetime`
5. Your server calls that function and returns the result
6. Model now has enough information to provide the final answer

We will need a tool use loop until the model stop calling tools

In [ ]:
def run_conversation(query):
    messages = [
        {
            "role": "system",
            "content":"""You are a helpful calendar bot.
You have access to three tools (get_current_datetime, set_reminder_schema, add_duration_to_datetime) to help with user's requests."""
        },
        {
            "role":"user",
            "content": query
        }
    ]

    while True:
        response = client.chat.completions.create(
            model='google/gemini-2.5-flash',
            messages=messages,
            tools=calendar_tools,
        )
        llm_response = response.choices[0].message
        print(f"{llm_response}")

        if response.choices[0].finish_reason != 'tool_calls':
            print(f"\nAssistant: {llm_response.content}")
            messages.append({'role':'assistant','content':llm_response.content})
            break

        messages.append(llm_response)
        tool_results = run_tools(llm_response.tool_calls)
        messages.extend(tool_results)

In [ ]:
query = "What day is 103 days from today?"

run_conversation(query)

ChatCompletionMessage(content='', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='tool_0_get_current_datetime', function=Function(arguments='{"date_format":"%Y-%m-%d"}', name='get_current_datetime'), type='function', index=0)], reasoning=None)
ChatCompletionMessage(content='', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='tool_0_add_duration_to_datetime', function=Function(arguments='{"datetime_str":"2025-07-28","input_format":"%Y-%m-%d","duration":103,"unit":"days"}', name='add_duration_to_datetime'), type='function', index=0)], reasoning=None)
ChatCompletionMessage(content='103 days from today will be Saturday, November 08, 2025.', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None, reasoning=None)

Assistant: 103 days from today will be Saturday, November 08, 2025.


In [ ]:
query = "Set a reminder for my doctors appointment. Its 177 days after Jan 1st, 2050."
run_conversation(query)

ChatCompletionMessage(content='', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='tool_0_add_duration_to_datetime', function=Function(arguments='{"datetime_str":"2050-01-01","duration":177,"unit":"days"}', name='add_duration_to_datetime'), type='function', index=0)], reasoning=None)
ChatCompletionMessage(content='', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='tool_0_set_reminder', function=Function(arguments='{"content":"Doctors Appointment","timestamp":"2050-06-27T00:00:00"}', name='set_reminder'), type='function', index=0)], reasoning=None)
----
Setting the following reminder for 2050-06-27T00:00:00:
Doctors Appointment
----
ChatCompletionMessage(content="OK. I've set a reminder for your doctor's appointment on Monday, June 27, 2050 12:00:00 AM.\n", refusal=None, role='assistant', annotations=None, audio=None, function_cal